<a href="https://colab.research.google.com/github/nanfenggushi/colab-all-in-one-downloader/blob/main/colab_all_in_one_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 我的专属全能云端下载器 (最终极限速度版)

**功能:**
- **HTTP/HTTPS 直链下载**: 使用多线程 `aria2c`，已优化参数以达到最大速度。
- **BitTorrent 磁力链接下载**: 使用 `libtorrent`，采用激进的连接和缓存策略以最大化下载速度。
- **YouTube 及流媒体视频下载**: 使用 `yt-dlp`，增加并发分片下载以提速。

**使用流程:**
1.  **从上到下，依次运行所有代码单元格**。
2.  在【步骤2】完成后，您需要根据提示完成 Google Drive 的授权。
3.  在【步骤3】的图形化界面中，选择对应的下载类型，粘贴链接，确认保存路径，然后点击下载按钮。
4.  下载任务开始后，您可以关闭页面，稍后直接在您的 Google Drive 中查看文件。

In [ ]:
# ===================================================================
# 步骤 1: 安装所有必需的工具 (终极优化版)
# ===================================================================
print("🚀 开始安装所有必需的下载工具...")

# 1. 使用 apt-get 在系统层面安装 aria2c 命令行工具
print("正在安装 aria2c 命令行工具...")
!apt-get -qq update
!apt-get -qq install -y aria2

# 2. 使用 pip 安装 libtorrent 和其他Python库
print("正在安装 libtorrent, yt-dlp 等Python库...")
!pip -q install libtorrent yt-dlp

# 3. 安装后自检，确保关键工具都已就位
print("\n--- 安装后自检 ---")
!which aria2c
try:
    import libtorrent
    print(f"libtorrent-ok, version: {libtorrent.version}")
except ImportError as e:
    print(f"❌ libtorrent 导入失败: {e}")
!which yt-dlp
print("--------------------")

print("\n✅ 所有工具安装完毕！")

In [ ]:
# ===================================================================
# 步骤 2: 连接Google Drive并准备核心功能 (终极速度优化版)
# ===================================================================
print("📚 正在导入库并准备核心功能 (已针对终极速度优化)...")

# --- 核心库导入 ---
import os
import time
import libtorrent as lt
from google.colab import drive
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 连接 Google Drive ---
print("🔗 正在请求连接到您的 Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
    print("✅ Google Drive 连接成功！")
except Exception as e:
    print(f"❌ Google Drive 连接失败: {e}")

# --- 创建临时缓存目录 ---
!mkdir -p /content/temp_downloads

# --- 定义HTTP下载函数 (终极速度优化) ---
def start_http_download(links, save_path):
    os.makedirs(save_path, exist_ok=True)
    temp_path = "/content/temp_downloads"
    os.makedirs(temp_path, exist_ok=True)
    print(f"⬇️ 开始HTTP下载 (终极速度模式)，临时缓存: {temp_path} → 最终保存: {save_path}")
    link_list = [link.strip() for link in links.strip().split('\n') if link.strip()]
    if not link_list:
        print("⚠️ 链接列表为空。")
        return

    # 终极速度参数:
    # -x 16: 单个服务器最大连接数 (Colab上限通常是16)
    # -s 16: 所有任务总并发连接数
    # -k 10M: 最小分片大小，增大到10M，减少管理开销
    # --optimize-concurrent-downloads=true: 自动优化并发下载设置
    # --auto-file-renaming=false: 避免文件重命名的开销
    # --file-allocation=none: 减少文件分配的时间
    aria2_options = "-c -x 16 -s 16 -k 10M --optimize-concurrent-downloads=true --auto-file-renaming=false --file-allocation=none --max-concurrent-downloads=20 --summary-interval=0 --allow-overwrite=true"

    command = f'nohup aria2c {aria2_options} -d "{temp_path}"'
    for link in link_list:
        command += f' "{link}"'
    command += f' > /content/aria2_log.txt 2>&1 && rsync -av --progress "{temp_path}/" "{save_path}/" && rm -rf "{temp_path}"/* &'

    get_ipython().system(command)
    print(f"\n🎉 {len(link_list)} 个HTTP下载任务已提交到后台处理。")
    print("ℹ️ 下载先缓存到临时目录，完成后自动移至Google Drive。")
    print("ℹ️ 您可以继续添加其他任务或关闭页面。")

# --- 定义BT下载函数 (终极速度优化) ---
def start_bt_download(magnet_link, save_path):
    os.makedirs(save_path, exist_ok=True)
    temp_path = "/content/temp_downloads/bt_temp"
    os.makedirs(temp_path, exist_ok=True)
    print(f"🧲 开始BT下载 (终极速度模式)，临时缓存: {temp_path} → 最终保存: {save_path}")

    if not magnet_link.startswith("magnet:?"):
        print("⚠️ 无效的磁力链接。")
        return

    # 终极速度设置:
    settings = {
        'user_agent': 'qBittorrent/4.4.5',
        'listen_interfaces': '0.0.0.0:6881',
        'peer_turnover': 300,             # 更频繁的Peer替换
        'max_peerlist_size': 4000,        # 大幅增加peer列表大小
        'connections_limit': 3000,        # 更高的总连接数上限
        'download_rate_limit': 0,         # 不限速
        'upload_rate_limit': 0,           # 不限速
        'cache_size': 4096,               # 增大缓存(4096 * 16KB = 64MB)
        'announce_to_all_trackers': True, # 向所有tracker宣告
        'announce_to_all_tiers': True,    # 向所有层级宣告
        'alert_mask': lt.alert.category_t.all_categories
    }

    # 添加常用优质tracker
    additional_trackers = [
        "udp://tracker.opentrackr.org:1337/announce",
        "udp://open.tracker.cl:1337/announce",
        "udp://9.rarbg.com:2810/announce",
        "udp://tracker.openbittorrent.com:6969/announce",
        "http://tracker.openbittorrent.com:80/announce"
    ]

    ses = lt.session(settings)
    params = {
        'save_path': temp_path,
        'storage_mode': lt.storage_mode_t.storage_mode_sparse
    }

    try:
        handle = lt.add_magnet_uri(ses, magnet_link, params)
        # 添加额外的tracker
        for tracker in additional_trackers:
            handle.add_tracker({"url": tracker})
    except Exception as e:
        print(f"❌ 添加磁力链接失败: {e}")
        return

    print("正在获取种子元数据...")
    while not handle.has_metadata():
        time.sleep(1)
        if not handle.is_valid():
            print("❌ 磁力链接无效或无法获取元数据。")
            return

    print(f"元数据获取成功！种子名称: {handle.status().name}")
    handle.set_max_connections(800) # 提高单个种子连接数上限
    handle.set_max_uploads(-1) # 不限制上传连接数
    handle.set_sequential_download(False) # 使用并行而非顺序下载

    progress_bar = widgets.FloatProgress(value=0, min=0, max=100, description='下载中:')
    display(progress_bar)

    while handle.status().state != lt.torrent_status.seeding:
        s = handle.status()
        progress = s.progress * 100
        progress_bar.value = progress
        progress_bar.description = f"DL:{s.download_rate/1000/1000:.2f}MB/s, UL:{s.upload_rate/1000/1000:.2f}MB/s | Peers:{s.num_peers}/{s.list_peers} | {progress:.2f}%"
        time.sleep(1)

    progress_bar.bar_style = 'success'
    progress_bar.description = f"✅ 下载完成: {handle.status().name}"

    # 下载完成后移动到Google Drive
    torrent_name = handle.status().name
    print(f"\n正在将文件移动到Google Drive: {torrent_name}")
    get_ipython().system(f'rsync -av --progress "{temp_path}/{torrent_name}" "{save_path}/" && rm -rf "{temp_path}/{torrent_name}"')

    print(f"\n🎉 BT任务 '{torrent_name}' 下载完成并已移至Google Drive！")

# --- 定义YouTube下载函数 (终极速度优化) ---
def start_youtube_download(url, save_path):
    os.makedirs(save_path, exist_ok=True)
    temp_path = "/content/temp_downloads/yt_temp"
    os.makedirs(temp_path, exist_ok=True)
    print(f"📹 开始YouTube/视频下载 (终极速度模式)，临时缓存: {temp_path} → 最终保存: {save_path}")
    if not url.strip():
        print("⚠️ 请提供有效的视频链接。")
        return

    # 终极速度参数:
    # -N 16: 同时下载16个分块
    # --concurrent-fragments 20: 对于分片视频流，同时下载20个片段
    # --downloader aria2c: 使用aria2c作为下载器
    # --downloader-args: 传递下载器参数
    yt_dlp_options = "-N 16 --concurrent-fragments 20 --downloader aria2c --downloader-args 'aria2c:-x 16 -s 16 -k 10M'"

    # 先下载到临时目录，完成后移动到Google Drive
    command = f'cd "{temp_path}" && yt-dlp {yt_dlp_options} -f "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best" --merge-output-format mp4 "{url}" && rsync -av --progress "{temp_path}/"* "{save_path}/" && rm -rf "{temp_path}"/* > /dev/null 2>&1 &'
    get_ipython().system(command)
    print("\n🎉 视频下载任务已提交到后台处理。")
    print("ℹ️ 下载先缓存到临时目录，完成后自动移至Google Drive。")
    print("ℹ️ 您可以继续添加其他任务或关闭页面。")

print("✅ 核心功能已准备就绪！")

In [ ]:
# ===================================================================
# 步骤 3: 启动下载器图形界面 (GUI)
# ===================================================================

# --- 默认保存路径 ---
# 您可以永久修改这里的默认路径
default_save_path = '/content/drive/MyDrive/Downloads'

# --- 创建HTTP下载界面组件 ---
http_links_input = widgets.Textarea(
    placeholder='在此处输入一个或多个HTTP/HTTPS直链，每行一个。',
    layout={'height': '100px', 'width': '98%'}
)
http_save_path_input = widgets.Text(value=default_save_path, description='保存路径:', layout={'width': '98%'})
http_button = widgets.Button(description="开始HTTP下载", button_style='success', icon='download')
http_output = widgets.Output()

def on_http_button_clicked(b):
    with http_output:
        clear_output(wait=True)
        start_http_download(http_links_input.value, http_save_path_input.value)
http_button.on_click(on_http_button_clicked)
http_tab = widgets.VBox([widgets.Label(value="将HTTP/S直链文件下载到您的Google Drive。"), http_save_path_input, http_links_input, http_button, http_output])

# --- 创建BT下载界面组件 ---
bt_magnet_input = widgets.Text(placeholder='在此处输入一个磁力链接。', layout={'width': '98%'})
bt_save_path_input = widgets.Text(value=default_save_path, description='保存路径:', layout={'width': '98%'})
bt_button = widgets.Button(description="开始BT下载", button_style='info', icon='magnet')
bt_output = widgets.Output()

def on_bt_button_clicked(b):
    with bt_output:
        clear_output(wait=True)
        start_bt_download(bt_magnet_input.value, bt_save_path_input.value)
bt_button.on_click(on_bt_button_clicked)
bt_tab = widgets.VBox([widgets.Label(value="通过磁力链接下载文件到您的Google Drive。"), bt_save_path_input, bt_magnet_input, bt_button, bt_output])

# --- 创建YouTube下载界面组件 ---
yt_url_input = widgets.Text(placeholder='在此处输入一个YouTube或其他支持的视频链接。', layout={'width': '98%'})
yt_save_path_input = widgets.Text(value=default_save_path, description='保存路径:', layout={'width': '98%'})
yt_button = widgets.Button(description="开始视频下载", button_style='danger', icon='youtube')
yt_output = widgets.Output()

def on_yt_button_clicked(b):
    with yt_output:
        clear_output(wait=True)
        start_youtube_download(yt_url_input.value, yt_save_path_input.value)
yt_button.on_click(on_yt_button_clicked)
yt_tab = widgets.VBox([widgets.Label(value="从YouTube等网站下载视频到您的Google Drive。"), yt_save_path_input, yt_url_input, yt_button, yt_output])

# --- 创建状态监控组件 ---
monitor_button = widgets.Button(description="查看下载状态", button_style='warning', icon='dashboard')
monitor_output = widgets.Output()

def on_monitor_button_clicked(b):
    with monitor_output:
        clear_output(wait=True)
        print("📊 当前下载状态:")
        !ps aux | grep -E 'aria2c|yt-dlp' | grep -v grep
        print("\n📂 临时文件夹状态:")
        !du -sh /content/temp_downloads/* 2>/dev/null || echo "暂无临时下载文件"
monitor_button.on_click(on_monitor_button_clicked)
monitor_tab = widgets.VBox([widgets.Label(value="监控当前下载状态和临时文件"), monitor_button, monitor_output])

# --- 创建并显示选项卡界面 ---
tab = widgets.Tab()
tab.children = [http_tab, bt_tab, yt_tab, monitor_tab]
tab.set_title(0, '🔗 HTTP / HTTPS')
tab.set_title(1, '🧲 BitTorrent (磁力)')
tab.set_title(2, '📹 YouTube & More')
tab.set_title(3, '📊 下载监控')

print("✅ 终极下载器界面已准备好，请在下方操作。")
print("🔥 此版本已实现Colab环境下的理论极限速度优化")
display(tab)